**# Masoud Karami**      **2050716**


[INF8953CE(Fall 2020) : Machine Learning](http://sarathchandar.in/teaching/ml/fall2020/)
[Assignment #3](https://drive.google.com/file/d/1UNZ-ozmZ2OsG9L7XTzKBEud9A6aZaOgN/view?usp=sharing) Due on : Nov 16, 10:00 pm

* You have to submit the pdf copy of the report on gradescope before the deadline. If you
handwrite your solutions, you need to scan the pages, merge them to a single pdf file and
submit. Mark page 1 for outline items $Late\ Submission$ and $Verbosity$.

* When asked to report the parameters, save the corresponding parameters in a text file
(.txt) with the following template for the name: 
`Assignment1 <Matricule> <section> <sub-question > <sub-sub-question>`. Submit all the text files and codes compressed
to a single file `<your-matricule>.zip` on Moodle.

$Note:$ gradescope doesn’t accept `.zip`.

* Be precise with your explanations in the report. Unnecessary verbosity will be penalized.
* You are free to use libraries with general utilities, such as `matplotlib`, `numpy` and `scipy` for
`python`. You can use pre-existing implementation of the algorithms available in scikit-learn
package. Do not use NLTK or any other NLP libraries for pre-processing.!

---
---

# Sentiment Classification

In this assignment, we will design a sentiment classifier for classifying the sentiment of the
reviews. This is a *Natural Language Processing (NLP)* task where the input is a natural
language text and output is the sentiment label. We will consider two different review
datasets: Yelp reviews for restaurants and IMDB reviews for movies.

# Yelp dataset

The Yelp dataset consists of 7000 reviews in the training set, 1000 reviews in the validation
set, and 2000 reviews in the test set. This is a 5 class problem where each review is classified
into one of the five ratings with rating-5 being the best score and rating-1 being the worst
score.

# IMDB dataset

IMDB dataset consists of 15000 reviews in training set, 10000 reviews in validation set, and
25000 reviews in test set. This is a 2 class problem with class 1 being positive sentiment and
class 0 being negative sentiment.

In [3]:
# import sys
# import os


# filelist = [ f for f in os.listdir('/content/') if f.endswith(".txt") or f.endswith(".csv")]
# for f in filelist:
#     os.remove(os.path.join('/content/', f))
# sys.modules[__name__].__dict__.clear()

> # **1.** 
Most of the algorithms described in the class expects input as a vector. However, the
reviews are natural language text of varying number of words. So the first step would
be to convert this varying length movie review to a fixed length vector representation.
We will consider two different ways of vectorizing the natural language text: binary
bag-of-words representation and frequency bag-of-words representation (as explained in
the end of the assignment). Convert both the datasets into both these representations
and turn in the converted datasets. Instruction for dataset format is given in the end of
the assignment (do not include the dataset in the report).

In [4]:
# print('Question #1', '\n')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import string
import operator
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn import tree
from sklearn.svm import LinearSVC


data_path = '/content/'
Yelp_train = pd.read_csv(data_path + "yelp-train.txt", sep='\t', header=None)
Yelp_test = pd.read_csv(data_path + "yelp-test.txt", sep='\t', header=None)
Yelp_valid = pd.read_csv(data_path + "yelp-valid.txt", sep='\t', header=None)
IMDB_train = pd.read_csv(data_path + "IMDB-train.txt", sep='\t', header=None)
IMDB_test = pd.read_csv(data_path + "IMDB-test.txt", sep='\t', header=None)
IMDB_valid = pd.read_csv(data_path + "IMDB-valid.txt", sep='\t', header=None)


#  removal of punctuation
out = set(string.punctuation)
seperator = '#'*100


In [5]:

def bin_bag_of_words(data_set):
  counti = data_set[0].count()
  for counter in range(counti):
    senti = data_set[0][counter]
    senti = ''.join(char for char in senti if char not in out)
    senti = senti.lower()
    data_set[0][counter] = senti
  return data_set



In [6]:
Yelp_train = bin_bag_of_words(Yelp_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# binary bag of words
%%time

Yelp_train = bin_bag_of_words(Yelp_train)
Yelp_test = bin_bag_of_words(Yelp_test)
Yelp_valid = bin_bag_of_words(Yelp_valid)
IMDB_train = bin_bag_of_words(IMDB_train)
IMDB_test = bin_bag_of_words(IMDB_test)
IMDB_valid = bin_bag_of_words(IMDB_valid)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 23.8 s, sys: 21.8 ms, total: 23.8 s
Wall time: 24 s


In [8]:


Yelp_train = Yelp_train.values
Yelp_test = Yelp_test.values
Yelp_valid = Yelp_valid.values

IMDB_train = IMDB_train.values
IMDB_test = IMDB_test.values
IMDB_valid = IMDB_valid.values
# print(Yelp_train)
# print(IMDB_train)

In [9]:

def top_myriad(data_set):
  pre_data = {}
  for col in range(data_set.shape[0]):
    sep = data_set[col][0].split()
    for idems in range(len(sep)):
      if sep[idems] in pre_data:
        pre_data[sep[idems]] += 1
      else:
        pre_data[sep[idems]] = 1
  return pre_data

In [10]:



top_train_Yelp = []
top_train_IMDB = []
sortit_Yelp = top_myriad(Yelp_train).items()
sortit_IMDB = top_myriad(IMDB_train).items()
sort_train_Yelp = sorted(sortit_Yelp, key=operator.itemgetter(1), reverse=True)
sort_train_IMDB = sorted(sortit_IMDB, key=operator.itemgetter(1), reverse=True)


# only consider the training set
for i in range(10000):
  top_train_Yelp.append(sort_train_Yelp[i][0])
  top_train_IMDB.append(sort_train_IMDB[i][0])




In [11]:


def vector_bin_bag(data_set, n, Yelp = True):
  vector = np.zeros(shape=(data_set.shape[0], n), dtype=int)
  top_train = top_train_Yelp if Yelp else top_train_IMDB
  for col in range(data_set.shape[0]):
    mot = data_set[col][0].split(' ')
    for counter in range(n):
      if top_train[counter] in mot:
        vector[col][counter] = 1
      else:
        vector[col][counter] = 0
  return vector


In [12]:
# takes time, be patient please
%%time


Yelp_train_vec = vector_bin_bag(Yelp_train, 10000, True)
Yelp_test_vec = vector_bin_bag(Yelp_test, 10000, True)
Yelp_valid_vec = vector_bin_bag(Yelp_valid, 10000, True)


CPU times: user 3min 9s, sys: 544 ms, total: 3min 10s
Wall time: 3min 10s


In [13]:
class_Yelp_train = Yelp_train[:,1].astype(str)
class_Yelp_test = Yelp_test[:,1].astype(str)
class_Yelp_valid = Yelp_valid[:,1].astype(str)


In [14]:
# count the frequency
# sort based on frequency
# descending order
# pick top 10,000 words, ignore the rest


f = open(data_path + '#1-Yelp_mot.txt', 'w+', encoding='utf-8')
for counter in range(10000):
  f.write(sort_train_Yelp[counter][0] + '\t' + str(counter+1) + '\t' + 
          str(sort_train_Yelp[counter][1]) + '\n')
f.close()

In [15]:
# takes time, be patient please
%%time

IMDB_train_vec = vector_bin_bag(IMDB_train, 10000, False)
IMDB_test_vec = vector_bin_bag(IMDB_test, 10000, False)
IMDB_valid_vec = vector_bin_bag(IMDB_valid, 10000, False)

CPU times: user 25min 44s, sys: 2.75 s, total: 25min 47s
Wall time: 25min 48s


In [16]:
class_IMDB_train = IMDB_train[:,1].astype(str)
class_IMDB_test = IMDB_test[:,1].astype(str)
class_IMDB_valid = IMDB_valid[:,1].astype(str)

In [17]:
# count the frequency
# sort based on frequency
# descending order
# pick top 10,000 words, ignore the rest

g = open(data_path + '#1-IMDB_mot.txt', 'w+', encoding='utf-8')
for counter in range(10000):
  g.write(sort_train_IMDB[counter][0] + '\t' + str(counter+1) + '\t' + 
          str(sort_train_IMDB[counter][1]) + '\n')
g.close()

In [18]:
# generate dataset Yelp train

with open('#1-Yelp_train.txt', 'w', encoding='utf-8') as f:
  for col in range(Yelp_train.shape[0]):
    mot = Yelp_train[col][0].split(' ')
    temp_Yelp = [(top_train_Yelp.index(verb)+1) if verb in top_train_Yelp else verb for verb in mot]
    for item in temp_Yelp:
      f.write("%s " %item)
    f.write('\t' + class_Yelp_train[col] + '\n')
f.close()


In [19]:
%%time
# generate dataset Yelp valid

with open('#1-Yelp_valid.txt', 'w', encoding='utf-8') as g:
  for col in range(Yelp_valid.shape[0]):
    mot = Yelp_valid[col][0].split(' ')
    temp_Yelp = [(top_train_Yelp.index(verb)+1) if verb in top_train_Yelp else verb for verb in mot]
    for item in temp_Yelp:
      if item == temp_Yelp[-1]:
        g.write("%s" %item)
      g.write("%s " %item)
    g.write('\t' + class_Yelp_valid[col] + '\n')
g.close()

CPU times: user 3.77 s, sys: 7.99 ms, total: 3.78 s
Wall time: 3.79 s


In [20]:
%%time
# generate dataset Yelp test

with open('#1-Yelp_test.txt', 'w', encoding='utf-8') as h:
  for col in range(Yelp_test.shape[0]):
    mot = Yelp_test[col][0].split(' ')
    temp_Yelp = [(top_train_Yelp.index(verb)+1) if verb in top_train_Yelp else verb for verb in mot]
    for item in temp_Yelp:
      if item == temp_Yelp[-1]:
        h.write("%s" %item)
      h.write("%s " %item)
      h.write('\t' + class_Yelp_test[col] + '\n')
h.close()

CPU times: user 7.38 s, sys: 29 ms, total: 7.41 s
Wall time: 7.42 s


In [21]:
# generate dataset IMDB train
%%time


with open('#1-IMDB_train.txt', 'w', encoding='utf-8') as m:
  for col in range(IMDB_train.shape[0]):
    mot = IMDB_train[col][0].split(' ')
    temp_IMDB = [(top_train_IMDB.index(verb)+1) if verb in top_train_IMDB else verb for verb in mot]
    for item in temp_IMDB:
      m.write("%s " %item)
    m.write('\t' + class_IMDB_train[col] + '\n')
m.close()

CPU times: user 1min 57s, sys: 123 ms, total: 1min 57s
Wall time: 1min 57s


In [22]:
%%time
# generate dataset IMDB valid

with open('#1-IMDB_valid.txt', 'w', encoding='utf-8') as n:
  for col in range(IMDB_valid.shape[0]):
    mot = IMDB_valid[col][0].split(' ')
    temp_IMDB = [(top_train_IMDB.index(verb)+1) if verb in top_train_IMDB else verb for verb in mot]
    for item in temp_IMDB:
      if item == temp_IMDB[-1]:
        n.write("%s" %item)
      n.write("%s " %item)
      n.write('\t' + class_IMDB_valid[col] + '\n')
n.close()

CPU times: user 1min 20s, sys: 127 ms, total: 1min 20s
Wall time: 1min 20s


In [23]:
%%time
# generate dataset IMDB test

with open('#1-IMDB_test.txt', 'w', encoding='utf-8') as k:
  for col in range(IMDB_test.shape[0]):
    mot = IMDB_test[col][0].split(' ')
    temp_IMDB = [(top_train_IMDB.index(verb)+1) if verb in top_train_IMDB else verb for verb in mot]
    for item in temp_IMDB:
      if item == temp_IMDB[-1]:
        k.write("%s" %item)
      k.write("%s " %item)
      k.write('\t' + class_IMDB_test[col] + '\n')
k.close()

CPU times: user 3min 14s, sys: 297 ms, total: 3min 14s
Wall time: 3min 14s


># **2.**
For this question, we will focus on the Yelp dataset with binary bag-of-words (BBoW)
representation. We will use the F1-measure as the evaluation metric for the entire
assignment.







>> # **2.(a)** 
As a baseline, report the performance of the random classifier (a classifier which
classifies a review into an uniformly random class) and the majority-class classifier
(a classifier which computes the majority class in the training set and classifies all
test instances as that majority class).

In [24]:
# print('Question #2.a', '\n')
# performance of the random classifier

def uniform_rand_classifier(data_set, data_class, file_name):
  rand = np.empty(shape = (data_class.shape[0]), dtype=int)
  for col in range(data_set.shape[0]):
    if 'Yelp' in file_name:
      rand[col] = np.random.randint(1, 6)
    else:
      rand[col] = np.random.randint(0, 2)
  rand = rand.astype(str)
  f1_random = metrics.f1_score(data_class, rand, average='macro')
  print ('BBoW F1 for random classifier on {} = {}'.format(file_name, f1_random))
  if 'Yelp' in file_name:
    f = open("#2_a-Ranadclass_perf.txt", 'a+')
    f.write('BBoW F1 for random classifier on {} = {} \n'.format(file_name, f1_random))
    f.close()
  else:
    f = open("#4_a-Randclass_perf.txt", 'a+')
    f.write('BBoW F1 for random classifier on {} = {} \n'.format(file_name, f1_random))
    f.close()
  return f1_random, rand


In [25]:
%%time


f1_Yelp_train, rand_Yelp_train = uniform_rand_classifier(Yelp_train, class_Yelp_train, 'Yelp_train')
f1_Yelp_test, rand_Yelp_test = uniform_rand_classifier(Yelp_test, class_Yelp_test, 'Yelp_test')
f1_Yelp_valid, rand_Yelp_valid = uniform_rand_classifier(Yelp_valid, class_Yelp_valid, 'Yelp_valid')

BBoW F1 for random classifier on Yelp_train = 0.18241916362346142
BBoW F1 for random classifier on Yelp_test = 0.18057797685118898
BBoW F1 for random classifier on Yelp_valid = 0.17749389657271505
CPU times: user 73.1 ms, sys: 1 ms, total: 74.1 ms
Wall time: 75 ms


In [26]:
# np.savetxt('#2-a-BBoW_Yelp_train_rand_classifier.csv', rand_Yelp_train, fmt='%s')
# np.savetxt('#2-a-BBoW_Yelp_valid_rand_classifier.csv', rand_Yelp_valid, fmt='%s')
# np.savetxt('#2-a-BBoW_Yelp_test_rand_classifier.csv', rand_Yelp_test, fmt='%s')

In [27]:
# performance of the majority-class classifier
# IMDB is balanced, so majority doesn't make sense

def majority_classifier(data_set, data_class, file_name):
  list_major = {}
  for col in range(Yelp_train.shape[0]):
    if Yelp_train[col][1] in list_major:
      list_major[Yelp_train[col][1]] += 1
    else:
      list_major[data_set[col][1]] = 1
  majority = max(list_major.items(), key = operator.itemgetter(1))[0]
  majority_out = np.full((data_set.shape[0], 1), majority)
  majority_out = np.squeeze(majority_out).astype(str)
  f1_majority = metrics.f1_score(data_class, majority_out, average='micro')
  print('BBoW majority class for {} is {} \n'.format(file_name[0:4], majority))
  print('BBoW F1 for majority classifier on {} = {}'.format(file_name, f1_majority))
  f = open("#2_a-Majority_Perf.txt", 'a+')
  f.write('BBoW majority class for {} is {}. \n F1 for majority classifier on {} = {}.\n'.
          format(file_name[0:4], majority, file_name, f1_majority))
  f.close()
  return f1_majority

In [28]:
%%time


f1_Yelp_train_majority = majority_classifier(Yelp_train, class_Yelp_train, 'Yelp_train')
f1_Yelp_test_majority = majority_classifier(Yelp_test, class_Yelp_test, 'Yelp_test')
f1_Yelp_valid_majority = majority_classifier(Yelp_valid, class_Yelp_valid, 'Yelp_valid')


BBoW majority class for Yelp is 4 

BBoW F1 for majority classifier on Yelp_train = 0.3525714285714286
BBoW majority class for Yelp is 4 

BBoW F1 for majority classifier on Yelp_test = 0.351
BBoW majority class for Yelp is 4 

BBoW F1 for majority classifier on Yelp_valid = 0.356
CPU times: user 49.8 ms, sys: 1.99 ms, total: 51.8 ms
Wall time: 52.3 ms


>> # **2.(b)**
Now train Naive Bayes, Decision Trees, and Linear SVM for this task. [Note: You
should do a thorough hyper-parameter tuning by using the given validation set.
Also, note that you should use the appropriate naive Bayes classifier for binary
input features (also called Bernoulli naive Bayes).]


In [29]:
# print('Question #2.b', '\n')
#  naive Bayes classifier or Bernoulli naive Bayes
%%time

from sklearn.naive_bayes import BernoulliNB
import numpy as np

def NB_BBoW(train_vec, ctrain, cvalid, valid_vec, Yelp = True):
  alphas = [0.0001, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 
            0.007, 0.008, 0.009, 0.01, 0.02, 0.05, 0.1]
  pred_NB = []
  f1_NB = []
  f = open("#2_c-NB_perform.txt", 'a+')
  g = open("#4_c-NB_perform.txt", 'a+')
  for i in range(len(alphas)):
    NB_Class = BernoulliNB(alpha=alphas[i],binarize=None)
    NB_Class.fit(train_vec, ctrain)
    test = NB_Class.predict(train_vec)
    pred_NB.append(NB_Class.predict(valid_vec))
    F1_train = metrics.f1_score(ctrain, test, average='macro')
    F1_valid = metrics.f1_score(cvalid, pred_NB[i], average='macro')
    print("BBoW----alpha:{},  F1_train: {},  F1_valid: {}".
          format(alphas[i], F1_train, F1_valid))
    f1_NB.append(metrics.f1_score(cvalid, pred_NB[i], average='macro'))
    if Yelp:
      f.write("BBoW----alpha:{},  F1_train: {},  F1_valid: {} \n".
              format(alphas[i], F1_train, F1_valid))
    else:
      g.write("BBoW----alpha:{},  F1_train: {},  F1_valid: {} \n".
              format(alphas[i], F1_train, F1_valid))
  alpha_optimum = alphas[f1_NB.index(max(f1_NB))]
  f.close()
  g.close()
  return f1_NB

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 16.9 µs


In [30]:
nb_bbow_Y = NB_BBoW(Yelp_train_vec, class_Yelp_train, class_Yelp_valid, Yelp_valid_vec, True)

BBoW----alpha:0.0001,  F1_train: 0.7913623590092731,  F1_valid: 0.3342684884698385
BBoW----alpha:0.001,  F1_train: 0.7848232083669805,  F1_valid: 0.3577699545850232
BBoW----alpha:0.002,  F1_train: 0.7816064001267711,  F1_valid: 0.36014365009756233
BBoW----alpha:0.003,  F1_train: 0.779424995491092,  F1_valid: 0.36328782357848977
BBoW----alpha:0.004,  F1_train: 0.7780436311699405,  F1_valid: 0.37185227307442653
BBoW----alpha:0.005,  F1_train: 0.7770036545680943,  F1_valid: 0.3752693102868807
BBoW----alpha:0.006,  F1_train: 0.7756761352138442,  F1_valid: 0.376461283228757
BBoW----alpha:0.007,  F1_train: 0.7734679454609646,  F1_valid: 0.37804555182992194
BBoW----alpha:0.008,  F1_train: 0.772313747740285,  F1_valid: 0.37819556762594175
BBoW----alpha:0.009,  F1_train: 0.7713498093412052,  F1_valid: 0.3769282700440043
BBoW----alpha:0.01,  F1_train: 0.7703866381707726,  F1_valid: 0.37643923150496417
BBoW----alpha:0.02,  F1_train: 0.761571272639455,  F1_valid: 0.3766558313116145
BBoW----alpha:0

In [31]:
# decision trees classifier 
%%time

from sklearn import tree

def DecisiontreeBBoW(train_vec, valid_vec, ctrain, cvalid, Yelp = True):
  depth = [8,9,10,11,12, 15, 20, 25, 30, 40, 50]
  criterias =  ['gini', 'entropy']
  pred_tree = []
  f1_tree = []
  f = open("#2_c-tree_perform.txt", 'a+')
  g = open("#4_c-tree_perform.txt", 'a+')
  for crit in criterias:
    for i in range(len(depth)):
      tree_Class = tree.DecisionTreeClassifier(criterion = crit, max_depth=depth[i])
      tree_Class.fit(train_vec, ctrain)
      test = tree_Class.predict(train_vec)
      pred_tree.append(tree_Class.predict(valid_vec))
      F1_train = metrics.f1_score(ctrain, test, average='macro')
      F1_valid = metrics.f1_score(cvalid, pred_tree[i], average='macro')
      print("Depth:{}, criterion:{}, F1_train: {},  F1_valid: {}".
            format(depth[i], crit, F1_train, F1_valid))
      f1_tree.append(metrics.f1_score(cvalid, pred_tree[i], average='macro'))
      if Yelp:
        f.write("Depth:{}, criterion:{}, F1_train: {},  F1_valid: {} \n".
                format(depth[i], crit, F1_train, F1_valid))
      else:
        g.write("Depth:{}, criterion:{}, F1_train: {},  F1_valid: {} \n".
                format(depth[i], crit, F1_train, F1_valid))        
  f.close()
  g.close()

CPU times: user 35 µs, sys: 0 ns, total: 35 µs
Wall time: 22.9 µs


In [32]:
DecisiontreeBBoW(Yelp_train_vec, Yelp_valid_vec, class_Yelp_train, class_Yelp_valid, True)

Depth:8, criterion:gini, F1_train: 0.3572480170206753,  F1_valid: 0.2769670002521454
Depth:9, criterion:gini, F1_train: 0.39538205028665196,  F1_valid: 0.28102294301858377
Depth:10, criterion:gini, F1_train: 0.4239427588506867,  F1_valid: 0.2836796920981451
Depth:11, criterion:gini, F1_train: 0.46048886104435277,  F1_valid: 0.2748731312927169
Depth:12, criterion:gini, F1_train: 0.5095300168038713,  F1_valid: 0.2622851792710646
Depth:15, criterion:gini, F1_train: 0.6180853795730803,  F1_valid: 0.2686567345330662
Depth:20, criterion:gini, F1_train: 0.7481147741456773,  F1_valid: 0.27425524554464586
Depth:25, criterion:gini, F1_train: 0.85275844868159,  F1_valid: 0.2917560154190291
Depth:30, criterion:gini, F1_train: 0.9112112771666594,  F1_valid: 0.28057014295808547
Depth:40, criterion:gini, F1_train: 0.9669765629956526,  F1_valid: 0.2636514784789812
Depth:50, criterion:gini, F1_train: 0.992782033304796,  F1_valid: 0.2768311187089814
Depth:8, criterion:entropy, F1_train: 0.36415778802774

In [33]:
# Linear SVM for Yelp
%%time

from warnings import filterwarnings
from sklearn.svm import LinearSVC
filterwarnings('ignore')


def svm_BBoW(train_vec, ctrain, valid_vec, cvalid, Yelp = True):
  C_svm = [2**-4, 2**-3, 2**-2, 0.5, 1, 1.5, 2, 4,10.0, 2**4, 2**5]
  pred_svm = []
  f1_svm = []
  f = open("#2_c-svm_perform.txt", 'a+')
  g = open("#4_c-svm_perform.txt", 'a+')
  for i in range(len(C_svm)):
    svm_Class = LinearSVC(C = C_svm[i], multi_class="ovr", loss='hinge')
    svm_Class.fit(train_vec, ctrain)
    test = svm_Class.predict(train_vec)
    pred_svm.append(svm_Class.predict(valid_vec))
    F1_train = metrics.f1_score(ctrain, test, average='macro')
    F1_valid = metrics.f1_score(cvalid, pred_svm[i], average='macro')
    print("C = {}, F1_train = {},  F1_valid = {}".
          format(C_svm[i], F1_train, F1_valid))
    f1_svm.append(F1_valid)
    if Yelp:
      f.write("Depth C = {}, F1_train = {},  F1_valid = {} \n".
              format(C_svm[i], F1_train, F1_valid))
    else:
      g.write("Depth C = {}, F1_train = {},  F1_valid = {} \n".
              format(C_svm[i], F1_train, F1_valid))
  f.close()
  g.close()

CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 59.1 µs


In [34]:
svm_BBoW(Yelp_train_vec, class_Yelp_train, Yelp_valid_vec, class_Yelp_valid,  True)

C = 0.0625, F1_train = 0.8891018180003357,  F1_valid = 0.44214202372104605
C = 0.125, F1_train = 0.9330357708601534,  F1_valid = 0.4327375108805448
C = 0.25, F1_train = 0.9643105259111024,  F1_valid = 0.42830463326490326
C = 0.5, F1_train = 0.9821937373467182,  F1_valid = 0.4258720518688831
C = 1, F1_train = 0.9915682725894561,  F1_valid = 0.41895827342783587
C = 1.5, F1_train = 0.995078383919996,  F1_valid = 0.41437499937449845
C = 2, F1_train = 0.9968351781610512,  F1_valid = 0.4104237697675691
C = 4, F1_train = 0.9986588053911886,  F1_valid = 0.4116521116510574
C = 10.0, F1_train = 0.9987566871494623,  F1_valid = 0.4084241077910953
C = 16, F1_train = 0.9991048062789718,  F1_valid = 0.4070251582429811
C = 32, F1_train = 0.9993756543056056,  F1_valid = 0.40179086412858284


>> # **2.(c)**
 Report the list of hyper-parameters you considered for each classifier, the range
of the individual hyper-parameters and the best value for these hyper-parameters
chosen based on the validation set performance.




In [35]:
# F1 score for the optimal NB-BBoW hyperparameter 

alphas = [0.0001, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.05, 0.1]
alpha_optimum = alphas[nb_bbow_Y.index(max(nb_bbow_Y))]
print('The optimal alpha for the NB classifier is {}'.format(alpha_optimum))
f = open("#2_c-NB_parameters.txt", 'a+')
f.write("The optimal alpha from the set {} w.r.t NB_classifier is {} \n".
        format(alphas, alpha_optimum))
f.close()


def Yelp_optimal_f1(vector, data_class, file_name):
  NB_Class = BernoulliNB(alpha = alpha_optimum)
  NB_Class.fit(Yelp_train_vec, class_Yelp_train)
  predicted_result = (NB_Class.predict(vector))
  f1_result = metrics.f1_score(data_class, predicted_result, average='macro')
  print('NB_BBoW best values for {} with optimal hyperparameters is {}'.
        format(file_name, f1_result))
  f = open("#2_c-NB_parameters.txt", 'a+')
  f.write('{} \n NB_BBoW best values for {} with optimal hyperparameters is {}. \n'.
        format(seperator, file_name, f1_result))



The optimal alpha for the NB classifier is 0.008


In [36]:
from sklearn.dummy import DummyClassifier as DC
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn import svm, metrics
from sklearn.metrics import f1_score

In [37]:


seperator = '#'*100
NB_params = ParameterGrid({'alpha':[0.001, 0.002, 0.003, 0.004, 0.005, 
                                    0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.05, 0.1]})
tree_params = ParameterGrid({'criterion':['gini','entropy'],
                             'max_features': [None, "sqrt", "log2"],
                             'max_depth':[8, 9, 10, 15, 20, 30, 50, 100],
                             'min_samples_leaf': [0.2, 0.49],
                             'min_samples_split':[2, 3, 5, 10, 20]})
svm_params = ParameterGrid({'loss':['hinge','squared_hinge'],
                            'C':[0.25, 0.5, 1.5, 2, 5, 10, 100]})

classifiers = [(BernoulliNB, NB_params), (DTC, tree_params), (svm.LinearSVC, svm_params)]

In [38]:
def model_validation(model):
  model.fit(Yelp_train_vec, class_Yelp_train)
  valid = model.predict(Yelp_valid_vec)
  f1_valid = f1_score(class_Yelp_valid, valid, average = 'micro')
  return f1_valid

###########################################################################

def optimize_parameters(classifier, grid, Yelp = True):
  if Yelp:
    f = open("#2_c-All Parameters.txt", 'a+')
    best_f1 = 0.0
    optimal_params = None
    for params in grid:
      f1 = model_validation(classifier(**params))
      f.write("{}-----> F1 = {}\n".format(params, f1))
      print("{}-----> F1 = {}".format(params, f1))
      if f1 > best_f1:
          best_f1 = f1
          optimal_params = params       
    print("Best params: {}".format(optimal_params))
    print("Best F1 Score: {}\n {}".format(best_f1, seperator))
    f.write("Best params: {}\n Best F1 Score = {}\n{}\n".
            format(optimal_params, best_f1, seperator))
    f.close()
  else:
    g = open("#4_c-All Parameters.txt", 'a+')
    best_f1 = 0.0
    optimal_params = None
    for params in grid:
      f1 = model_validation_4(classifier(**params))
      g.write("{} --------> F1 = {}\n".format(params, f1))
      print("{} --------> F1 = {}".format(params, f1))
      if f1 > best_f1:
          best_f1 = f1
          optimal_params = params       
    print("Best params: {}".format(optimal_params))
    print("Best F1 Score: {}\n {} \n".format(best_f1, seperator))
    g.write("Best params: {}\n Best F1 Score = {}\n{}\n".
            format(optimal_params, best_f1, seperator))
    g.close()    
  return classifier(**optimal_params)

###########################################################################

def model_train_valid_test(model):
  model.fit(Yelp_train_vec, class_Yelp_train)
  train = model.predict(Yelp_train_vec)
  valid = model.predict(Yelp_valid_vec)
  test = model.predict(Yelp_test_vec)
  f1_train = f1_score(class_Yelp_train, train, average='micro')
  f1_valid = f1_score(class_Yelp_valid, valid, average='micro')
  f1_test = f1_score(class_Yelp_test, test, average='micro')
  return f1_train, f1_valid, f1_test


In [39]:
%%time


uni_random = DC(strategy = 'uniform')
majority_c = DC(strategy = 'most_frequent')

print('Random Classifier = {:<25}'.format(model_validation(uni_random)))
print('Majority Classifier = {:<25}'.format(model_validation(majority_c)))

for param in classifiers:
    clf = param[0]
    param_grid = param[1]
    print(clf)    
    best_clf = optimize_parameters(clf, param_grid)

Random Classifier = 0.202                    
Majority Classifier = 0.356                    
<class 'sklearn.naive_bayes.BernoulliNB'>
{'alpha': 0.001}-----> F1 = 0.416
{'alpha': 0.002}-----> F1 = 0.414
{'alpha': 0.003}-----> F1 = 0.415
{'alpha': 0.004}-----> F1 = 0.41999999999999993
{'alpha': 0.005}-----> F1 = 0.421
{'alpha': 0.006}-----> F1 = 0.421
{'alpha': 0.007}-----> F1 = 0.421
{'alpha': 0.008}-----> F1 = 0.421
{'alpha': 0.009}-----> F1 = 0.419
{'alpha': 0.01}-----> F1 = 0.419
{'alpha': 0.02}-----> F1 = 0.421
{'alpha': 0.05}-----> F1 = 0.413
{'alpha': 0.1}-----> F1 = 0.40700000000000003
Best params: {'alpha': 0.005}
Best F1 Score: 0.421
 ####################################################################################################
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
{'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 0.2, 'min_samples_split': 2}-----> F1 = 0.38499999999999995
{'criterion': 'gini', 'max_depth': 8, 'max_features

>> # **2.(d)**
Report training, validation, and test F1-measure for all the classifiers (with best
hyper-parameter configuration).

In [40]:
%%time

Yelp_train_F1_NB_optparam = Yelp_optimal_f1(Yelp_train_vec, class_Yelp_train, 'Yelp_train')
Yelp_test_F1_NB_optparam = Yelp_optimal_f1(Yelp_test_vec, class_Yelp_test, 'Yelp_test')
Yelp_valid_F1_NB_optparam = Yelp_optimal_f1(Yelp_valid_vec, class_Yelp_valid, 'Yelp_valid')

NB_BBoW best values for Yelp_train with optimal hyperparameters is 0.772313747740285
NB_BBoW best values for Yelp_test with optimal hyperparameters is 0.35768974964514666
NB_BBoW best values for Yelp_valid with optimal hyperparameters is 0.37819556762594175
CPU times: user 7 s, sys: 334 ms, total: 7.34 s
Wall time: 6.88 s


In [41]:
import re
from sklearn.tree import DecisionTreeClassifier

params = []
with open("#2_c-All Parameters.txt") as f:
  for line in f:
    if 'Best params:' in line:
      params.append(line)
temp1 = repr(params[0])
temp1 = re.sub('Best params: ', '', temp1)
NB_best_param = float(temp1.split()[1][:-4])
with open("#2_d-Best_Results.txt", 'a+') as g:
  g.write('NB_best alpha = {}\n'.format(NB_best_param))
print('NB_best alpha = {}'.format(NB_best_param))
#################################################


NB_best alpha = 0.005


In [42]:
temp2 = repr(params[1])
temp2 = re.sub('Best params: ', '', temp2)
criterion = temp2.split()[1][:-1]
criterion = criterion.replace("'",'')
max_dep = [int(s) for s in temp2.split()[3] if s.isdigit()][0]
max_feat = temp2.split()[5][:-1]
if max_feat==str(None):
  max_feat = None
else:
  max_feat = max_feat.replace("'", '')
min_leaf = float(temp2.split()[7][:-1])
mss = int(temp2.split()[9][:-4])
tree_best_params = [criterion, max_dep, max_feat, min_leaf, mss]
with open("#2_d-Best_Results.txt", 'a+') as g:
  g.write('[criterion, max_depth, max_features, min_samples_leaf, min_samples_split] = {}\n'.
          format(tree_best_params))
print('criterion, max_depth, max_features, min_samples_leaf, min_samples_split =', tree_best_params)
##################################################


criterion, max_depth, max_features, min_samples_leaf, min_samples_split = ['gini', 8, None, 0.2, 2]


In [43]:
temp3 = repr(params[2])
temp3 = re.sub('Best params: ', '', temp3)
c = float(temp3.split()[1][:-1])
temp3_loss = temp3.split()[3][:-4]
temp3_loss = temp3_loss.replace("'","" )
svm_best_params = [c, temp3_loss]
with open("#2_d-Best_Results.txt", 'a+') as g:
  g.write('[C, loss] = {}\n {}'.
          format(svm_best_params, seperator))
print('[C, loss] = ', svm_best_params)

[C, loss] =  [0.25, 'squared_hinge']


In [44]:
print('NB_best_params are {},\n tree_best_params are {},\n svm_best_params are {}.'.
      format(NB_best_param, tree_best_params, svm_best_params))



NB_best_params are 0.005,
 tree_best_params are ['gini', 8, None, 0.2, 2],
 svm_best_params are [0.25, 'squared_hinge'].


In [45]:
optimized_bayes = BernoulliNB(alpha = NB_best_param)
optimized_tree = DecisionTreeClassifier(criterion = tree_best_params[0],
                                        max_features = tree_best_params[2],
                                        max_depth = float(tree_best_params[1]),
                                        min_samples_leaf = tree_best_params[3],
                                        min_samples_split = tree_best_params[4], 
                                        random_state = 42)
optimized_svm = svm.LinearSVC(C = svm_best_params[0], loss = svm_best_params[1], random_state = 42)


In [46]:
%%time

print('Optimal NB f1 for train, valid, test = {}'.format(model_train_valid_test(optimized_bayes)))
print('Optimal Tree f1 for train, valid, test = {}'.format(model_train_valid_test(optimized_tree)))
print('Optimal SVM f1 for train, valid, test = {}'.format(model_train_valid_test(optimized_svm)))

f = open("#2_d-Best_Results.txt", 'a+')
f.write('\n NB_best_params are {}, \n tree_best_params are {}, \n svm_best_params are {} \n {}\n'.
        format(NB_best_param, tree_best_params, svm_best_params, seperator))
f.write('Optimal NB f1 for the (train, valid, test) = {},\n Optimal Tree f1 for train, valid, test = {}, \n Optimal SVM f1 for train, valid, test = {}\n {}'.
        format(model_train_valid_test(optimized_bayes), model_train_valid_test(optimized_tree), model_train_valid_test(optimized_svm), seperator))
f.close()

Optimal NB f1 for train, valid, test = (0.7544285714285714, 0.421, 0.432)
Optimal Tree f1 for train, valid, test = (0.39614285714285713, 0.38499999999999995, 0.38499999999999995)
Optimal SVM f1 for train, valid, test = (0.9857142857142858, 0.466, 0.4595)
CPU times: user 16.4 s, sys: 774 ms, total: 17.1 s
Wall time: 15.5 s


>>**2.(e)**
Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyperparameter that fetched you the best results.


> # **3**
Now we will repeat question 2 but with frequency bag-of-words (FBoW) representation.



>> # **3.(a)**

Train Naive Bayes, Decision Trees, and Linear SVM for this task. [Note: You should do a thorough hyper-parameter tuning by using the given validation set. Also, note that you should use the appropriate naive Bayes classifier for real valued input features (also called Gaussian naive Bayes).]

In [47]:
 print('Question #3.a', '\n')
# random classifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import string
import operator
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn import tree
from sklearn.svm import LinearSVC


data_path = '/content/'
out = set(string.punctuation)
df_train_3 = pd.read_csv(data_path + 'yelp-train.txt', sep='\t', header=None)
df_test_3 = pd.read_csv(data_path + 'yelp-test.txt', sep='\t', header=None)
df_valid_3 = pd.read_csv(data_path + 'yelp-valid.txt', sep='\t', header=None)

def bin_bag_of_words(data_set):
  counti = data_set[0].count()
  for counter in range(counti):
    senti = data_set[0][counter]
    senti = ''.join(char for char in senti if char not in out)
    senti = senti.lower()
    data_set[0][counter] = senti
  return data_set


def top_myriad(data_set):
  pre_data = {}
  for col in range(data_set.shape[0]):
    sep = data_set[col][0].split()
    for idems in range(len(sep)):
      if sep[idems] in pre_data:
        pre_data[sep[idems]] += 1
      else:
        pre_data[sep[idems]] = 1
  return pre_data



Question #3.a 



In [48]:
%%time

FBoW_freq_Yelp_train = bin_bag_of_words (df_train_3)
FBoW_freq_Yelp_test = bin_bag_of_words (df_test_3)
FBoW_freq_Yelp_valid = bin_bag_of_words (df_valid_3)

CPU times: user 2.07 s, sys: 3.98 ms, total: 2.07 s
Wall time: 2.08 s


In [49]:
%%time

FBoW_freq_Yelp_train = df_train_3.values
FBoW_freq_Yelp_test = df_test_3.values
FBoW_freq_Yelp_valid = df_valid_3.values

CPU times: user 1.76 ms, sys: 0 ns, total: 1.76 ms
Wall time: 1.92 ms


In [50]:

top_train_Yelp_FBoW = []
sortit_Yelp_FBoW = top_myriad(FBoW_freq_Yelp_train).items()
sort_train_Yelp_FBoW = sorted(sortit_Yelp_FBoW, key=operator.itemgetter(1), reverse=True)
for i in range(10000):
  top_train_Yelp_FBoW.append(sort_train_Yelp_FBoW[i][0])

In [51]:
# frequency bag-of-words for train, validation, test
%%time


def frequency_classifier(data_set, file_name, n, top):
  rand = np.zeros(shape = (data_set.shape[0], n))
  for col in range(data_set.shape[0]):
    mot = data_set[col][0].split(' ')
    for counter in range(n):
      rand[col][counter] = mot.count(top[counter])
    if (np.sum(rand[col]) == 0):
      continue
    rand[col] = rand[col]/(np.sum(rand[col]))
  return rand

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.4 µs


In [52]:
%%time

freq_Yelp_train_vec = frequency_classifier(FBoW_freq_Yelp_train, 'FBoW_Yelp_train', 
                                           10000, top_train_Yelp_FBoW)
freq_Yelp_test_vec = frequency_classifier(FBoW_freq_Yelp_test, 'FBoW_Yelp_test', 
                                          10000, top_train_Yelp_FBoW)
freq_Yelp_valid_vec = frequency_classifier(FBoW_freq_Yelp_valid, 'FBoW_Yelp_valid', 
                                           10000, top_train_Yelp_FBoW)


CPU times: user 3min 56s, sys: 42.9 ms, total: 3min 56s
Wall time: 3min 57s


In [53]:
%%time 

class_Yelp_train_freq = FBoW_freq_Yelp_train[:,1].astype(str)
class_Yelp_valid_freq = FBoW_freq_Yelp_valid[:,1].astype(str)
class_Yelp_test_freq = FBoW_freq_Yelp_test[:,1].astype(str)


CPU times: user 3.36 ms, sys: 0 ns, total: 3.36 ms
Wall time: 3.68 ms


In [54]:
# random classifier
%%time

def FBoW_rand_classifier(data_set, data_class, file_name, n):
  rand = np.empty(shape=(data_class.shape[0]), dtype=int)
  for col in range(data_set.shape[0]):
    rand[col] = np.random.randint(1, 6)
  rand = rand.astype(str)
  f1_random = metrics.f1_score(data_class, rand, average='macro')
  print ('FBoW F1 for random classifier on {} = {}'.format(file_name, f1_random))
  f = open("#3_b-Parameters.txt", 'a+')
  f.write('FBoW F1 for random classifier on {} = {} \n'.format(file_name, f1_random))
  f.close()
  return f1_random, rand

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [55]:
%%time
rand_Yelp_train_FBoW = FBoW_rand_classifier(FBoW_freq_Yelp_train, class_Yelp_train_freq, 
                                            'Yelp_train', 10000)
rand_Yelp_test_FBoW = FBoW_rand_classifier(FBoW_freq_Yelp_test, class_Yelp_test_freq, 
                                           'Yelp_test', 10000)
rand_Yelp_valid_FBoW = FBoW_rand_classifier(FBoW_freq_Yelp_valid, class_Yelp_valid_freq, 
                                            'Yelp_valid', 10000)



FBoW F1 for random classifier on Yelp_train = 0.1748819994496678
FBoW F1 for random classifier on Yelp_test = 0.1868315195588058
FBoW F1 for random classifier on Yelp_valid = 0.16450123957211815
CPU times: user 71.6 ms, sys: 999 µs, total: 72.6 ms
Wall time: 72.6 ms


In [56]:
# majority classifier
%%time


def majority_freq_class(data_set, data_class, file_name):
  list_major = {}
  for col in range(FBoW_freq_Yelp_train.shape[0]):
    if FBoW_freq_Yelp_train[col][1] in list_major:
      list_major[FBoW_freq_Yelp_train[col][1]] += 1
    else:
      list_major[data_set[col][1]] = 1
  majority = max(list_major.items(), key = operator.itemgetter(1))[0]
  majority_out = np.full((data_set.shape[0], 1), majority)
  majority_out = np.squeeze(majority_out).astype(str)
  f1_majority = metrics.f1_score(data_class, majority_out, average='micro')
  print('FBoW majority class for {} is {} \n'.format(file_name[0:4], majority))
  print('FBoW F1 for majority classifier on {} = {}'.format(file_name, f1_majority))
  f = open("#3_b-Parameters.txt", 'a+')
  f.write('BBoW majority class for {} is {}. \n F1 for majority classifier on {} = {}.\n'.
          format(file_name[0:4], majority, file_name, f1_majority))
  f.close()
  return f1_majority



CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [57]:
f1_Y_train_major_FBoW = majority_freq_class(FBoW_freq_Yelp_train, class_Yelp_train_freq, 'Yelp_train')
f1_Y_test_major_FBoW = majority_freq_class(FBoW_freq_Yelp_test, class_Yelp_test_freq, 'Yelp_test')
f1_Y_valid_major_FBoW = majority_freq_class(FBoW_freq_Yelp_valid, class_Yelp_valid_freq, 'Yelp_valid')

FBoW majority class for Yelp is 4 

FBoW F1 for majority classifier on Yelp_train = 0.3525714285714286
FBoW majority class for Yelp is 4 

FBoW F1 for majority classifier on Yelp_test = 0.351
FBoW majority class for Yelp is 4 

FBoW F1 for majority classifier on Yelp_valid = 0.356


In [58]:
# NB classifier
## Gaussian Bayes multiclass classifier

%%time

from sklearn.naive_bayes import GaussianNB

def NB_FBoW(vec, cfreq, file_name, Yelp = True):
  f1_NB = []
  g = open("#5_b-NB_perform.txt", 'a+')
  f = open("#3_b-Parameters.txt", 'a+')
  gs_clf = GaussianNB()
  gs_clf.fit(freq_Yelp_train_vec,class_Yelp_train_freq)
  y_pred = (gs_clf.predict(vec))
  f1_NB_FBoW = metrics.f1_score(cfreq, y_pred, average='macro')
  print('F1 score FBoW for Gaussian NB with optimal hyperparameter on {} = {}'.
        format(file_name, f1_NB_FBoW))
  if Yelp:
    f.write('FBoW NB F1 with optimal params for {} = {}\n'.format(file_name, f1_NB_FBoW))
    f.close()
  else:
    g.write('FBoW NB F1 with optimal params for {} = {}\n'.format(file_name, f1_NB_FBoW))
    g.close()
  return f1_NB_FBoW

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16.7 µs


In [59]:
freq_Yelp_train_f1NB = NB_FBoW(freq_Yelp_train_vec, class_Yelp_train_freq, 'training', True)
freq_Yelp_test_f1NB = NB_FBoW(freq_Yelp_test_vec, class_Yelp_test_freq, 'test set', True)
freq_Yelp_valid_f1NB = NB_FBoW(freq_Yelp_valid_vec, class_Yelp_valid_freq, 'validation', True)

NB_FBoW_opt = max

F1 score FBoW for Gaussian NB with optimal hyperparameter on training = 0.7878899542894812
F1 score FBoW for Gaussian NB with optimal hyperparameter on test set = 0.2477846680185553
F1 score FBoW for Gaussian NB with optimal hyperparameter on validation = 0.24561901520780366


In [60]:
%%time
# decision trees classifier 

from sklearn import tree


def Decisiontree_FBoW(train_vec, valid_vec, ctrain, cvalid, Yelp = True):
  depth_FBoW = [8,9,10,11,12, 15, 20, 25, 30, 40, 50]
  criterias =  ['gini', 'entropy']
  pred_tree = []
  f1_tree = []
  f = open("#3_b-tree_perform.txt", 'a+')
  g = open("#5_b-tree_perform.txt", 'a+')
  for crit in criterias:
    for i in range(len(depth_FBoW)):
      tree_Class = tree.DecisionTreeClassifier(criterion = crit, max_depth=depth_FBoW[i])
      tree_Class.fit(train_vec, ctrain)
      test = tree_Class.predict(train_vec)
      pred_tree.append(tree_Class.predict(valid_vec))
      F1_train = metrics.f1_score(ctrain, test, average='macro')
      F1_valid = metrics.f1_score(cvalid, pred_tree[i], average='macro')
      print("Depth:{:<4}, criterion:{:<10}, F1_train: {:<22},  F1_valid: {:<22}".
            format(depth_FBoW[i], crit, F1_train, F1_valid))
      f1_tree.append(metrics.f1_score(cvalid, pred_tree[i], average='macro'))
      if Yelp:
        f.write("Depth:{:<4}, criterion:{:<10}, F1_train: {:<22},  F1_valid: {:>22} \n".
                format(depth_FBoW[i], crit, F1_train, F1_valid))
      else:
        g.write("Depth:{:<4}, criterion:{:<10}, F1_train: {:<22},  F1_valid: {:>22} \n".
                format(depth_FBoW[i], crit, F1_train, F1_valid))        
  f.close()
  g.close()

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16 µs


In [61]:
Decisiontree_FBoW(freq_Yelp_train_vec, freq_Yelp_valid_vec, 
                 class_Yelp_train_freq, class_Yelp_valid_freq, True)

Depth:8   , criterion:gini      , F1_train: 0.3766426025958901    ,  F1_valid: 0.3043314347780416    
Depth:9   , criterion:gini      , F1_train: 0.4088290500488256    ,  F1_valid: 0.30171444240828793   
Depth:10  , criterion:gini      , F1_train: 0.4365026211186314    ,  F1_valid: 0.30651721623382205   
Depth:11  , criterion:gini      , F1_train: 0.47368042334067245   ,  F1_valid: 0.30755745695537295   
Depth:12  , criterion:gini      , F1_train: 0.5129415340358963    ,  F1_valid: 0.31050664358228447   
Depth:15  , criterion:gini      , F1_train: 0.6199766949086325    ,  F1_valid: 0.301878831600186     
Depth:20  , criterion:gini      , F1_train: 0.7716368837726384    ,  F1_valid: 0.30850546789608146   
Depth:25  , criterion:gini      , F1_train: 0.8841168871746234    ,  F1_valid: 0.2865171715358428    
Depth:30  , criterion:gini      , F1_train: 0.9461279984061545    ,  F1_valid: 0.30184595818391574   
Depth:40  , criterion:gini      , F1_train: 0.9953622704579057    ,  F1_valid: 0.2

In [62]:
%%time
# Linear SVM for Yelp

from warnings import filterwarnings
from sklearn.svm import LinearSVC
filterwarnings('ignore')


def svm_FBoW(train_vec, ctrain, valid_vec, cvalid, Yelp = True):
  C_svm = [2**-4, 2**-3, 2**-2, 0.5, 1, 1.5, 2, 4,10.0, 2**4, 2**5]
  pred_svm = []
  f1_svm = []
  f = open("#3_b-svm_perform.txt", 'a+')
  g = open("#5_b-svm_perform.txt", 'a+')
  for i in range(len(C_svm)):
    svm_Class = LinearSVC(C = C_svm[i], multi_class="ovr", loss='hinge')
    svm_Class.fit(train_vec, ctrain)
    test = svm_Class.predict(train_vec)
    pred_svm.append(svm_Class.predict(valid_vec))
    F1_train = metrics.f1_score(ctrain, test, average='macro')
    F1_valid = metrics.f1_score(cvalid, pred_svm[i], average='macro')
    print("C = {:<8}, F1_train = {:<24},  F1_valid = {:<24}".
          format(C_svm[i], F1_train, F1_valid))
    f1_svm.append(F1_valid)
    if Yelp:
      f.write("C = {:<8}, F1_train = {:<24},  F1_valid = {:>24}\n".
              format(C_svm[i], F1_train, F1_valid))
    else:
      g.write("C = {:<8}, F1_train = {:<24},  F1_valid = {:>24}\n".
              format(C_svm[i], F1_train, F1_valid))
  f.close()
  g.close()


CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 35.3 µs


In [63]:
%%time
# Linear SVM for Yelp

svm_FBoW(freq_Yelp_train_vec, class_Yelp_train_freq, 
         freq_Yelp_valid_vec, class_Yelp_valid_freq, True)


C = 0.0625  , F1_train = 0.5444620790011439      ,  F1_valid = 0.35848994413293744     
C = 0.125   , F1_train = 0.5491658629030052      ,  F1_valid = 0.3549529140485509      
C = 0.25    , F1_train = 0.547133320386162       ,  F1_valid = 0.3615700781245077      
C = 0.5     , F1_train = 0.5459753414041526      ,  F1_valid = 0.3663504810961493      
C = 1       , F1_train = 0.5521687521017751      ,  F1_valid = 0.3676354237434708      
C = 1.5     , F1_train = 0.5586678140598245      ,  F1_valid = 0.36643100878089935     
C = 2       , F1_train = 0.5594202175760417      ,  F1_valid = 0.37191927886177323     
C = 4       , F1_train = 0.5797286096482276      ,  F1_valid = 0.3682813210657655      
C = 10.0    , F1_train = 0.6137305441700811      ,  F1_valid = 0.37581455735267627     
C = 16      , F1_train = 0.6418141849034367      ,  F1_valid = 0.3824881629439889      
C = 32      , F1_train = 0.6891221663226335      ,  F1_valid = 0.40858366832874704     
CPU times: user 35.2 s, sys: 1.0

>> # **3.(b)**
 Report the list of hyper-parameters you considered for each classifier, the range
of the individual hyper-parameters and the best value for these hyper-parameters
chosen based on the validation set performance.


In [64]:
from sklearn.dummy import DummyClassifier as DC
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn import svm, metrics
from sklearn.metrics import f1_score

In [65]:


seperator = '#'*100
NB_params = ParameterGrid({'alpha':[0.001, 0.002, 0.003, 0.004, 0.005, 
                                    0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.05, 0.1]})
tree_params = ParameterGrid({'criterion':['gini','entropy'],
                             'max_features': [None, "sqrt", "log2"],
                             'max_depth':[8, 9, 10, 15, 20, 30, 50, 100],
                             'min_samples_leaf': [0.2, 0.49],
                             'min_samples_split':[2, 3, 5, 10, 20]})
svm_params = ParameterGrid({'loss':['hinge','squared_hinge'],
                            'C':[0.25, 0.5, 1.5, 2, 5, 10, 100]})

classifiers = [(BernoulliNB, NB_params), (DTC, tree_params), (svm.LinearSVC, svm_params)]

In [66]:
def model_validation_FBoW_3(model):
  model.fit(freq_Yelp_train_vec, class_Yelp_train_freq)
  valid = model.predict(freq_Yelp_valid_vec)
  f1_valid = f1_score(class_Yelp_valid_freq, valid, average = 'micro')
  return f1_valid

###########################################################################

def optimize_parameters_FBoW(classifier, grid, Yelp = True):
  if Yelp:
    f = open("#3_b-All Parameters.txt", 'a+')
    best_f1 = 0.0
    optimal_params = None
    for params in grid:
      f1 = model_validation_FBoW_3(classifier(**params))
      f.write("{}-----> F1 = {}\n".format(params, f1))
      print("{}-----> F1 = {}".format(params, f1))
      if f1 > best_f1:
          best_f1 = f1
          optimal_params = params       
    print("Best params: {}".format(optimal_params))
    print("Best F1 Score: {}\n {}".format(best_f1, seperator))
    f.write("Best params: {}\n Best F1 Score = {}\n{}\n".format(optimal_params, best_f1, seperator))
    f.close()
  else:
    g = open("#5_b-All Parameters.txt", 'a+')
    best_f1 = 0.0
    optimal_params = None
    for params in grid:
      f1 = model_validation_FBoW_5(classifier(**params))
      g.write("{} --------> F1 = {}\n".format(params, f1))
      print("{} --------> F1 = {}".format(params, f1))
      if f1 > best_f1:
          best_f1 = f1
          optimal_params = params       
    print("Best params: {}".format(optimal_params))
    print("Best F1 Score: {}\n {} \n".format(best_f1, seperator))
    g.write("Best params: {}\n Best F1 Score = {}\n{}\n".format(optimal_params, best_f1, seperator))
    g.close()    
  return classifier(**optimal_params)

###########################################################################

def model_train_valid_test_FBoW_3(model):
  model.fit(freq_Yelp_train_vec, class_Yelp_train_freq)
  train = model.predict(freq_Yelp_train_vec)
  valid = model.predict(freq_Yelp_valid_vec)
  test = model.predict(freq_Yelp_test_vec)
  f1_train = f1_score(class_Yelp_train_freq, train, average='micro')
  f1_valid = f1_score(class_Yelp_valid_freq, valid, average='micro')
  f1_test = f1_score(class_Yelp_test_freq, test, average='micro')
  return f1_train, f1_valid, f1_test


In [67]:
%%time


uni_random = DC(strategy = 'uniform')
majority_c = DC(strategy = 'most_frequent')

print('Random Classifier = {}'.format(model_validation_FBoW_3(uni_random)))
print('Majority Classifier = {}'.format(model_validation_FBoW_3(majority_c)))

for param in classifiers:
    clf = param[0]
    param_grid = param[1]
    print(clf)    
    best_clf = optimize_parameters_FBoW(clf, param_grid)

Random Classifier = 0.213
Majority Classifier = 0.356
<class 'sklearn.naive_bayes.BernoulliNB'>
{'alpha': 0.001}-----> F1 = 0.416
{'alpha': 0.002}-----> F1 = 0.414
{'alpha': 0.003}-----> F1 = 0.415
{'alpha': 0.004}-----> F1 = 0.41999999999999993
{'alpha': 0.005}-----> F1 = 0.421
{'alpha': 0.006}-----> F1 = 0.421
{'alpha': 0.007}-----> F1 = 0.421
{'alpha': 0.008}-----> F1 = 0.421
{'alpha': 0.009}-----> F1 = 0.419
{'alpha': 0.01}-----> F1 = 0.419
{'alpha': 0.02}-----> F1 = 0.421
{'alpha': 0.05}-----> F1 = 0.413
{'alpha': 0.1}-----> F1 = 0.40700000000000003
Best params: {'alpha': 0.005}
Best F1 Score: 0.421
 ####################################################################################################
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
{'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 0.2, 'min_samples_split': 2}-----> F1 = 0.38499999999999995
{'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 0.2, 'min_s

>> # **3.(c)**
 Report training, validation, and test F1-measure for all the classifiers (with best
hyper-parameter configuration).

In [68]:
import re
from sklearn.tree import DecisionTreeClassifier

params_3 = []
with open("#3_b-All Parameters.txt") as f:
  for line in f:
    if 'Best params:' in line:
      params_3.append(line)
temp1_3 = repr(params_3[0])
temp1_3 = re.sub('Best params: ', '', temp1_3)
NB_best_param_3 = float(temp1_3.split()[1][:-4])
with open("#3_c-Best_Results.txt", 'a+') as g:
  g.write('NB_best alpha = {}\n'.format(NB_best_param_3))
print('NB_best alpha = {}'.format(NB_best_param_3))
#################################################


NB_best alpha = 0.005


In [69]:
temp2_3 = repr(params_3[1])
temp2_3 = re.sub('Best params: ', '', temp2_3)
criterion_3 = temp2_3.split()[1][:-1]
criterion_3 = criterion_3.replace("'",'')
max_dep_3 = [int(s) for s in temp2_3.split()[3] if s.isdigit()][0]
max_feat_3 = temp2_3.split()[5][:-1]
if max_feat_3 == str(None):
  max_feat_3 = None
else:
  max_feat_3 = max_feat_3.replace("'", '')
min_leaf_3 = float(temp2.split()[7][:-1])
mss_3 = int(temp2_3.split()[9][:-4])
tree_best_params_3 = [criterion_3, max_dep_3, max_feat_3, min_leaf_3, mss_3]
with open("#2_d-Best_Results.txt", 'a+') as g:
  g.write('[criterion, max_depth, max_features, min_samples_leaf, min_samples_split] = {}\n'.
          format(tree_best_params_3))
print('criterion, max_depth, max_features, min_samples_leaf, min_samples_split =', tree_best_params)
##################################################


criterion, max_depth, max_features, min_samples_leaf, min_samples_split = ['gini', 8, None, 0.2, 2]


In [70]:
temp3_3 = repr(params_3[2])
temp3_3 = re.sub('Best params: ', '', temp3_3)
c_3 = float(temp3_3.split()[1][:-1])
temp3_loss_3 = temp3_3.split()[3][:-4]
temp3_loss_3 = temp3_loss_3.replace("'","" )
svm_best_params_3 = [c_3, temp3_loss_3]
with open("#2_d-Best_Results.txt", 'a+') as g:
  g.write('[C, loss] = {}\n {}'.
          format(svm_best_params_3, seperator))
print('[C, loss] = ', svm_best_params_3)

[C, loss] =  [100.0, 'squared_hinge']


In [71]:
print('NB_best_params are {},\n tree_best_params are {},\n svm_best_params are {}.'.
      format(NB_best_param_3, tree_best_params_3, svm_best_params_3))



NB_best_params are 0.005,
 tree_best_params are ['gini', 8, None, 0.2, 2],
 svm_best_params are [100.0, 'squared_hinge'].


In [72]:
optimized_bayes_3 = BernoulliNB(alpha = NB_best_param_3)
optimized_tree_3 = DecisionTreeClassifier(criterion = tree_best_params_3[0],
                                        max_features = tree_best_params_3[2],
                                        max_depth = float(tree_best_params_3[1]),
                                        min_samples_leaf = tree_best_params_3[3],
                                        min_samples_split = tree_best_params_3[4], 
                                        random_state = 42)
optimized_svm_3 = svm.LinearSVC(C = svm_best_params_3[0], loss = svm_best_params_3[1], random_state = 42)


In [73]:
%%time

print('Optimal NB f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_3(optimized_bayes_3)))
print('Optimal Tree f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_3(optimized_tree_3)))
print('Optimal SVM f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_3(optimized_svm_3)))

f = open("#3_c-Best_Results.txt", 'a+')
f.write('\n NB_best_params are {}, \n tree_best_params are {}, \n svm_best_params are {} \n {}\n'.
        format(NB_best_param_3, tree_best_params_3, svm_best_params_3, seperator))
f.write('Optimal NB f1 for the (train, valid, test) = {},\n Optimal Tree f1 for train, valid, test = {}, \n Optimal SVM f1 for train, valid, test = {}\n {}'.
        format(model_train_valid_test_FBoW_3(optimized_bayes_3), 
               model_train_valid_test_FBoW_3(optimized_tree_3), 
               model_train_valid_test_FBoW_3(optimized_svm_3), 
               seperator))
f.close()

Optimal NB f1 for train, valid, test = (0.7544285714285714, 0.421, 0.432)
Optimal Tree f1 for train, valid, test = (0.39628571428571424, 0.38499999999999995, 0.38499999999999995)
Optimal SVM f1 for train, valid, test = (0.882, 0.499, 0.5095)
CPU times: user 29.9 s, sys: 915 ms, total: 30.8 s
Wall time: 28.7 s


>> # **3.(d)**
Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyperparameter that fetched you the best results.

>> # **3.(e)**
Compare the performance with the binary bag-of-words based classifiers. Why the
difference in performance? Give a brief explanation comparing BBoW Naive Bayes
and FBoW Naive Bayes and similarly for Decision Trees and Linear SVM.


>> # **3.(f)**
Which representation is better? Why?

> # **4**
 Now we will repeat questions 2 and 3 but with IMDB dataset. For this question we will
use IMDB dataset with BBoW representation.



>> # **4.(a)**
As a baseline, report the performance of the random classifier (a classifier which
classifies a review into an uniformly random class). Note that the IMDB dataset
is a balanced dataset. Hence majority-class classifier doesn’t make sense for this
dataset, and you can omit it for this question.

In [74]:
# was
f1_IMDB_train, rand_IMDB_train = uniform_rand_classifier(IMDB_train, class_IMDB_train, 'IMDB_train')
f1_IMDB_test, rand_IMDB_test = uniform_rand_classifier(IMDB_test, class_IMDB_test, 'IMDB_test')
f1_IMDB_valid, rand_IMDB_valid = uniform_rand_classifier(IMDB_valid, class_IMDB_valid, 'IMDB_valid')

BBoW F1 for random classifier on IMDB_train = 0.5012238362032857
BBoW F1 for random classifier on IMDB_test = 0.49745915782212785
BBoW F1 for random classifier on IMDB_valid = 0.49189938519825605


In [75]:
# np.savetxt('#4-a-BBoW_IMDB_train_rand_classifier.csv', rand_IMDB_train, fmt='%s')
# np.savetxt('#4-a-BBoW_IMDB_valid_rand_classifier.csv', rand_IMDB_valid, fmt='%s')
# np.savetxt('#4-a-BBoW_IMDB_test_rand_classifier.csv', rand_IMDB_test, fmt='%s')

>> # **4.(b)**
 Now train Naive Bayes, Decision Trees, and Linear SVM for this task. [Note: You
should do a thorough hyper-parameter tuning by using the given validation set.
Also, note that you should use the appropriate naive Bayes classifier for binary
input features (also called Bernoulli naive Bayes).]


In [76]:
# print('Question #4.b', '\n')
#  naive Bayes classifier or Bernoulli naive Bayes IMDB
%%time

nb_bbow_IMDB = NB_BBoW(IMDB_train_vec, class_IMDB_train, class_IMDB_valid, IMDB_valid_vec, False)

BBoW----alpha:0.0001,  F1_train: 0.8728375442036234,  F1_valid: 0.8426777215921546
BBoW----alpha:0.001,  F1_train: 0.8727711184784329,  F1_valid: 0.8430770249072167
BBoW----alpha:0.002,  F1_train: 0.8727711184784329,  F1_valid: 0.8430770249072167
BBoW----alpha:0.003,  F1_train: 0.8727711184784329,  F1_valid: 0.8431758867243427
BBoW----alpha:0.004,  F1_train: 0.8727711184784329,  F1_valid: 0.843075480543835
BBoW----alpha:0.005,  F1_train: 0.8727711184784329,  F1_valid: 0.843075480543835
BBoW----alpha:0.006,  F1_train: 0.8727711184784329,  F1_valid: 0.8431758867243427
BBoW----alpha:0.007,  F1_train: 0.8727711184784329,  F1_valid: 0.8431751024792695
BBoW----alpha:0.008,  F1_train: 0.8727711184784329,  F1_valid: 0.843075480543835
BBoW----alpha:0.009,  F1_train: 0.8727041803275917,  F1_valid: 0.843075480543835
BBoW----alpha:0.01,  F1_train: 0.8727041803275917,  F1_valid: 0.843075480543835
BBoW----alpha:0.02,  F1_train: 0.8725038766289896,  F1_valid: 0.8433751342363114
BBoW----alpha:0.05,  F

In [77]:
# decision trees classifier 
%%time

svm_BBoW(IMDB_train_vec, class_IMDB_train, IMDB_valid_vec, class_IMDB_valid,  False)

C = 0.0625, F1_train = 0.9738664989476644,  F1_valid = 0.8687994751979007
C = 0.125, F1_train = 0.9868666652074072,  F1_valid = 0.8607999498879819
C = 0.25, F1_train = 0.9953999994888888,  F1_valid = 0.8567999942719997
C = 0.5, F1_train = 0.9991333332986667,  F1_valid = 0.8463996067829933
C = 1, F1_train = 0.9998666666642964,  F1_valid = 0.8442990268689179
C = 1.5, F1_train = 1.0,  F1_valid = 0.8422991657625869
C = 2, F1_train = 1.0,  F1_valid = 0.8410999221389619
C = 4, F1_train = 1.0,  F1_valid = 0.8407998407998408
C = 10.0, F1_train = 1.0,  F1_valid = 0.8407998407998408
C = 16, F1_train = 1.0,  F1_valid = 0.8407998407998408
C = 32, F1_train = 1.0,  F1_valid = 0.8407998407998408
CPU times: user 40.9 s, sys: 2.18 s, total: 43.1 s
Wall time: 39 s


>> # **4.(c)**
Report the list of hyper-parameters you considered for each classifier, the range
of the individual hyper-parameters and the best value for these hyper-parameters
chosen based on the validation set performance.

In [78]:
# F1 score for the optimal NB-BBoW hyperparameter 

alphas = [0.0001, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 
          0.008, 0.009, 0.01, 0.02, 0.05, 0.1]
alpha_optimum = alphas[nb_bbow_IMDB.index(max(nb_bbow_IMDB))]
print('The optimal alpha for the NB classifier for IMDB is {}'.format(alpha_optimum))
f = open("#4_c-NB_parameters.txt", 'a+')
f.write("The optimal alpha from the set {} for the IMDB dataset w.r.t NB_classifier is {} \n".
        format(alphas, alpha_optimum))
f.close()


def IMDB_optimal_f1(vector, data_class, file_name):
  NB_Class = BernoulliNB(alpha = alpha_optimum)
  NB_Class.fit(IMDB_train_vec, class_IMDB_train)
  predicted_result = (NB_Class.predict(vector))
  f1_result = metrics.f1_score(data_class, predicted_result, average='macro')
  print('NB_BBoW best values for the IMDB {} with optimal hyperparameters is {}'.
        format(file_name, f1_result))
  f = open("#4_d-NB_performance.txt", 'a+')
  f.write('{} \n NB_BBoW best values for the {} of IMDB dataset with optimal hyperparameters is {}. \n'.
        format(seperator, file_name, f1_result))

The optimal alpha for the NB classifier for IMDB is 0.02


In [79]:
from sklearn.dummy import DummyClassifier as DC
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn import svm, metrics
from sklearn.metrics import f1_score

In [80]:


seperator = '#'*100
NB_params = ParameterGrid({'alpha':[0.001, 0.002, 0.003, 0.004, 0.005, 
                                    0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.05, 0.1]})
tree_params = ParameterGrid({'criterion':['gini','entropy'],
                             'max_features': [None, "sqrt", "log2"],
                             'max_depth':[8, 9, 10, 15, 20, 30, 50, 100],
                             'min_samples_leaf': [0.2, 0.49],
                             'min_samples_split':[2, 3, 5, 10, 20]})
svm_params = ParameterGrid({'loss':['hinge','squared_hinge'],
                            'C':[0.25, 0.5, 1.5, 2, 5, 10, 100]})

classifiers = [(BernoulliNB, NB_params), (DTC, tree_params), (svm.LinearSVC, svm_params)]

In [81]:
def model_validation_4(model):
  model.fit(IMDB_train_vec, class_IMDB_train)
  valid = model.predict(IMDB_valid_vec)
  f1_valid = f1_score(class_IMDB_valid, valid, average = 'micro')
  return f1_valid

###########################################################################

def model_train_valid_test_4(model):
  model.fit(IMDB_train_vec, class_IMDB_train)
  train = model.predict(IMDB_train_vec)
  valid = model.predict(IMDB_valid_vec)
  test = model.predict(IMDB_test_vec)
  f1_train = f1_score(class_IMDB_train, train, average='micro')
  f1_valid = f1_score(class_IMDB_valid, valid, average='micro')
  f1_test = f1_score(class_IMDB_test, test, average='micro')
  return f1_train, f1_valid, f1_test


In [82]:
%%time


uni_random = DC(strategy = 'uniform')
majority_c = DC(strategy = 'most_frequent')

print('Random Classifier = {}'.format(model_validation_4(uni_random)))
# print('Majority Classifier = {:<25}'.format(model_validation_4(majority_c)))

for param in classifiers:
    clf = param[0]
    param_grid = param[1]
    print(clf)    
    best_clf = optimize_parameters(clf, param_grid, False)

Random Classifier = 0.5005
<class 'sklearn.naive_bayes.BernoulliNB'>
{'alpha': 0.001} --------> F1 = 0.8431000000000001
{'alpha': 0.002} --------> F1 = 0.8431000000000001
{'alpha': 0.003} --------> F1 = 0.8432
{'alpha': 0.004} --------> F1 = 0.8431000000000001
{'alpha': 0.005} --------> F1 = 0.8431000000000001
{'alpha': 0.006} --------> F1 = 0.8432
{'alpha': 0.007} --------> F1 = 0.8432
{'alpha': 0.008} --------> F1 = 0.8431000000000001
{'alpha': 0.009} --------> F1 = 0.8431000000000001
{'alpha': 0.01} --------> F1 = 0.8431000000000001
{'alpha': 0.02} --------> F1 = 0.8434
{'alpha': 0.05} --------> F1 = 0.8433
{'alpha': 0.1} --------> F1 = 0.843
Best params: {'alpha': 0.02}
Best F1 Score: 0.8434
 #################################################################################################### 

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
{'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 0.2, 'min_samples_split': 2} --------> F1 = 0.6311
{'cri

>> # **4.(d)**
 Report training, validation, and test F1-measure for all the classifiers (with best
hyper-parameter configuration).


In [83]:
%%time

IMDB_train_F1_NB_optparam = IMDB_optimal_f1(IMDB_train_vec, class_IMDB_train, 'IMDB_train')
IMDB_test_F1_NB_optparam = IMDB_optimal_f1(IMDB_test_vec, class_IMDB_test, 'IMDB_test')
IMDB_valid_F1_NB_optparam = IMDB_optimal_f1(IMDB_valid_vec, class_IMDB_valid, 'IMDB_valid')

NB_BBoW best values for the IMDB IMDB_train with optimal hyperparameters is 0.8725038766289896
NB_BBoW best values for the IMDB IMDB_test with optimal hyperparameters is 0.835578653150356
NB_BBoW best values for the IMDB IMDB_valid with optimal hyperparameters is 0.8433751342363114
CPU times: user 16.3 s, sys: 3.17 s, total: 19.5 s
Wall time: 18.2 s


In [84]:
import re
from sklearn.tree import DecisionTreeClassifier

params_4 = []
with open("#4_c-All Parameters.txt") as f:
  for line in f:
    if 'Best params:' in line:
      params_4.append(line)
temp1_4 = repr(params_4[0])
temp1_4 = re.sub('Best params: ', '', temp1_4)
NB_best_param_4 = float(temp1_4.split()[1][:-4])
print('NB_best alpha IMDB = ',NB_best_param_4)
# #################################################


NB_best alpha IMDB =  0.02


In [85]:
temp2_4 = repr(params_4[1])
temp2_4 = re.sub('Best params: ', '', temp2_4)
criterion_4 = temp2_4.split()[1][:-1]
criterion_4 = criterion_4.replace("'",'')
max_dep_4 = [int(s) for s in temp2_4.split()[3] if s.isdigit()][0]
max_feat_4 = temp2_4.split()[5][:-1]
if max_feat_4==str(None):
  max_feat_4 = None
else:
  max_feat_4 = max_feat_4.replace("'", '')
min_leaf_4 = float(temp2_4.split()[7][:-1])
mss_4 = int(temp2_4.split()[9][:-4])
tree_best_params_4 = [criterion_4, max_dep_4, max_feat_4, min_leaf_4, mss_4]
print('criterion, max_depth, max_features, min_samples_leaf, min_samples_split =', 
      tree_best_params_4)
##################################################

criterion, max_depth, max_features, min_samples_leaf, min_samples_split = ['gini', 8, None, 0.2, 2]


In [86]:
temp3_4 = repr(params_4[2])
temp3_4 = re.sub('Best params: ', '', temp3_4)
print(temp3_4)
c_4 = float(temp3_4.split()[1][:-1])
temp3_loss_4 = temp3_4.split()[3][:-4]
temp3_loss_4 = temp3_loss_4.replace("'","" )
svm_best_params_4 = [c_4, temp3_loss_4]
print(svm_best_params_4)

"{'C': 0.25, 'loss': 'hinge'}\n"
[0.25, 'hinge']


In [87]:
print('NB_best_params are {},\n tree_best_params are {},\n svm_best_params are {}.'.
      format(NB_best_param_4, tree_best_params_4, svm_best_params_4))

NB_best_params are 0.02,
 tree_best_params are ['gini', 8, None, 0.2, 2],
 svm_best_params are [0.25, 'hinge'].


In [88]:
optimized_bayes_4 = BernoulliNB(alpha = NB_best_param_4)
optimized_tree_4 = DecisionTreeClassifier(criterion = tree_best_params_4[0],
                                        max_features = tree_best_params_4[2],
                                        max_depth = float(tree_best_params_4[1]),
                                        min_samples_split = tree_best_params_4[4], 
                                        random_state = 42)
optimized_svm_4 = svm.LinearSVC(C = svm_best_params_4[0], loss = svm_best_params_4[1], random_state = 42)


In [89]:
%%time

print('Optimal NB f1 for train, valid, test = {}'.format(model_train_valid_test_4(optimized_bayes_4)))
print('Optimal Tree f1 for train, valid, test = {}'.format(model_train_valid_test_4(optimized_tree_4)))
print('Optimal SVM f1 for train, valid, test = {}'.format(model_train_valid_test_4(optimized_svm_4)))

f = open("#4_d-Best_Results.txt", 'a+')
f.write('\n NB_best_params are {}, \n tree_best_params are {}, \n svm_best_params are {} \n {}\n'.
        format(NB_best_param_4, tree_best_params_4, svm_best_params_4, seperator))
f.write('Optimal NB f1 for (train, valid, test) = {},\n Optimal Tree f1 for train, valid, test = {}, \n Optimal SVM f1 for train, valid, test = {}\n {}'.
        format(model_train_valid_test_4(optimized_bayes_4), model_train_valid_test_4(optimized_tree_4), 
               model_train_valid_test_4(optimized_svm_4), seperator))
f.close()

Optimal NB f1 for train, valid, test = (0.8725333333333334, 0.8434, 0.83572)
Optimal Tree f1 for train, valid, test = (0.7364666666666666, 0.7088, 0.70664)
Optimal SVM f1 for train, valid, test = (0.9954, 0.8568, 0.83912)
CPU times: user 1min 31s, sys: 3.26 s, total: 1min 35s
Wall time: 1min 31s


>> # **4.(e)**
Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyperparameter that fetched you the best results.


># **5**
Now we will consider IMDB dataset with frequency bag-of-words (FBoW) representation.



>> # **5.(a)**
Train Naive Bayes, Decision Trees, and Linear SVM for this task. [Note: You
should do a thorough hyper-parameter tuning by using the given validation set.
Also, note that you should use the appropriate naive Bayes classifier for real valued
features (also called Gaussian naive Bayes).]

In [90]:
print('Question #5', '\n')
# random classifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import string
import operator
from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn import tree
from sklearn.svm import LinearSVC


data_path = '/content/'
out = set(string.punctuation)
df_train_5 = pd.read_csv(data_path + 'IMDB-train.txt', sep='\t', header=None)
df_test_5 = pd.read_csv(data_path + 'IMDB-test.txt', sep='\t', header=None)
df_valid_5 = pd.read_csv(data_path + 'IMDB-valid.txt', sep='\t', header=None)

def bin_bag_of_words(data_set):
  counti = data_set[0].count()
  for counter in range(counti):
    senti = data_set[0][counter]
    senti = ''.join(char for char in senti if char not in out)
    senti = senti.lower()
    data_set[0][counter] = senti
  return data_set


def top_myriad(data_set):
  pre_data = {}
  for col in range(data_set.shape[0]):
    sep = data_set[col][0].split()
    for idems in range(len(sep)):
      if sep[idems] in pre_data:
        pre_data[sep[idems]] += 1
      else:
        pre_data[sep[idems]] = 1
  return pre_data

Question #5 



In [91]:
%%time

FBoW_freq_IMDB_train = bin_bag_of_words(df_train_5)
FBoW_freq_IMDB_test = bin_bag_of_words(df_test_5)
FBoW_freq_IMDB_valid = bin_bag_of_words(df_valid_5)

CPU times: user 17.8 s, sys: 6.03 ms, total: 17.9 s
Wall time: 17.9 s


In [92]:
%%time

FBoW_freq_IMDB_train = df_train_5.values
FBoW_freq_IMDB_test = df_test_5.values
FBoW_freq_IMDB_valid = df_valid_5.values

CPU times: user 5.18 ms, sys: 0 ns, total: 5.18 ms
Wall time: 7.77 ms


In [93]:

top_train_IMDB_FBoW = []
sortit_IMDB_FBoW = top_myriad(FBoW_freq_IMDB_train).items()
sort_train_IMDB_FBoW = sorted(sortit_IMDB_FBoW, key=operator.itemgetter(1), reverse=True)
for i in range(10000):
  top_train_IMDB_FBoW.append(sort_train_IMDB_FBoW[i][0])

In [94]:
# frequency bag-of-words for train, validation, test
%%time


def frequency_classifier(data_set, file_name, n, top):
  rand = np.zeros(shape = (data_set.shape[0], n))
  for col in range(data_set.shape[0]):
    mot = data_set[col][0].split(' ')
    for counter in range(n):
      rand[col][counter] = mot.count(top[counter])
    if (np.sum(rand[col]) == 0):
      continue
    rand[col] = rand[col]/(np.sum(rand[col]))
  return rand

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.8 µs


In [95]:
%%time
# 30 min

freq_IMDB_train_vec = frequency_classifier(FBoW_freq_IMDB_train, 'FBoW_IMDB_train', 
                                           10000, top_train_IMDB_FBoW)
freq_IMDB_test_vec = frequency_classifier(FBoW_freq_IMDB_test, 'FBoW_IMDB_test', 
                                          10000, top_train_IMDB_FBoW)
freq_IMDB_valid_vec = frequency_classifier(FBoW_freq_IMDB_valid, 'FBoW_IMDB_valid', 
                                           10000, top_train_IMDB_FBoW)


CPU times: user 31min 39s, sys: 327 ms, total: 31min 39s
Wall time: 31min 40s


In [96]:
%%time 

class_IMDB_train_freq = FBoW_freq_IMDB_train[:,1].astype(str)
class_IMDB_valid_freq = FBoW_freq_IMDB_valid[:,1].astype(str)
class_IMDB_test_freq = FBoW_freq_IMDB_test[:,1].astype(str)


CPU times: user 9.72 ms, sys: 2 µs, total: 9.72 ms
Wall time: 9.64 ms


In [97]:
# random classifier
%%time

def FBoW_rand_class_IMDB(data_set, data_class, file_name, n):
  rand = np.empty(shape=(data_class.shape[0]), dtype=int)
  for col in range(data_set.shape[0]):
    rand[col] = np.random.randint(0, 2)
  rand = rand.astype(str)
  f1_random = metrics.f1_score(data_class, rand, average='macro')
  print ('FBoW F1 for random classifier on {} = {}'.format(file_name, f1_random))
  f = open("#5_b-Parameters.txt", 'a+')
  f.write('FBoW F1 for random classifier on {} = {} \n'.format(file_name, f1_random))
  f.close()
  return f1_random, rand

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.5 µs


In [98]:
%%time
rand_IMDB_train_FBoW = FBoW_rand_class_IMDB(FBoW_freq_IMDB_train, class_IMDB_train_freq, 
                                            'IMDB_train', 10000)
rand_IMDB_test_FBoW = FBoW_rand_class_IMDB(FBoW_freq_IMDB_test, class_IMDB_test_freq, 
                                           'IMDB_test', 10000)
rand_IMDB_valid_FBoW = FBoW_rand_class_IMDB(FBoW_freq_IMDB_valid, class_IMDB_valid_freq, 
                                            'IMDB_valid', 10000)



FBoW F1 for random classifier on IMDB_train = 0.5015891457191735
FBoW F1 for random classifier on IMDB_test = 0.500420297375856
FBoW F1 for random classifier on IMDB_valid = 0.49829634258033745
CPU times: user 388 ms, sys: 11 ms, total: 399 ms
Wall time: 407 ms


In [99]:
# decision trees classifier 
%%time

Decisiontree_FBoW(freq_IMDB_train_vec, freq_IMDB_valid_vec, 
                 class_IMDB_train_freq, class_IMDB_valid_freq, False)

Depth:8   , criterion:gini      , F1_train: 0.7334388309499673    ,  F1_valid: 0.6935889029918799    
Depth:9   , criterion:gini      , F1_train: 0.7544986062819226    ,  F1_valid: 0.7126169271076601    
Depth:10  , criterion:gini      , F1_train: 0.7633977428748029    ,  F1_valid: 0.7055268451027157    
Depth:11  , criterion:gini      , F1_train: 0.7862343143906685    ,  F1_valid: 0.714204944808578     
Depth:12  , criterion:gini      , F1_train: 0.8024558391958667    ,  F1_valid: 0.718407056122859     
Depth:15  , criterion:gini      , F1_train: 0.8404516261946074    ,  F1_valid: 0.7110261890756656    
Depth:20  , criterion:gini      , F1_train: 0.8860759493670886    ,  F1_valid: 0.7152537313684064    
Depth:25  , criterion:gini      , F1_train: 0.921694179477582     ,  F1_valid: 0.7094021715728833    
Depth:30  , criterion:gini      , F1_train: 0.9467974605669164    ,  F1_valid: 0.7081712062256809    
Depth:40  , criterion:gini      , F1_train: 0.9725835880484028    ,  F1_valid: 0.7

In [100]:
%%time
# Linear SVM for IMDB

svm_FBoW(freq_IMDB_train_vec, class_IMDB_train_freq, 
         freq_IMDB_valid_vec, class_IMDB_valid_freq, False)



C = 0.0625  , F1_train = 0.4523646735130934      ,  F1_valid = 0.4485251579948628      
C = 0.125   , F1_train = 0.48984009216123714     ,  F1_valid = 0.48512689106545165     
C = 0.25    , F1_train = 0.6525290044936152      ,  F1_valid = 0.6523174416791098      
C = 0.5     , F1_train = 0.6780600864740172      ,  F1_valid = 0.6757050054536926      
C = 1       , F1_train = 0.717221725147022       ,  F1_valid = 0.7082461011695428      
C = 1.5     , F1_train = 0.7405830778990883      ,  F1_valid = 0.7260484878562159      
C = 2       , F1_train = 0.7573643671703434      ,  F1_valid = 0.7412624556479059      
C = 4       , F1_train = 0.7959328991093676      ,  F1_valid = 0.7814576605852463      
C = 10.0    , F1_train = 0.839418420823323       ,  F1_valid = 0.8218720613893723      
C = 16      , F1_train = 0.8591027486104218      ,  F1_valid = 0.8371240085774418      
C = 32      , F1_train = 0.8817155349384482      ,  F1_valid = 0.8553477805487781      
CPU times: user 19.3 s, sys: 1.7

>> # **5.(b)**
Report the list of hyper-parameters you considered for each classifier, the range
of the individual hyper-parameters and the best value for these hyper-parameters
chosen based on the validation set performance.

In [101]:
def model_validation_FBoW_5(model):
  model.fit(freq_IMDB_train_vec, class_IMDB_train_freq)
  valid = model.predict(freq_IMDB_valid_vec)
  f1_valid = f1_score(class_IMDB_valid_freq, valid, average = 'micro')
  return f1_valid


def model_train_valid_test_FBoW_5(model):
  model.fit(freq_IMDB_train_vec, class_IMDB_train_freq)
  train = model.predict(freq_IMDB_train_vec)
  valid = model.predict(freq_IMDB_valid_vec)
  test = model.predict(freq_IMDB_test_vec)
  f1_train = f1_score(class_IMDB_train_freq, train, average='micro')
  f1_valid = f1_score(class_IMDB_valid_freq, valid, average='micro')
  f1_test = f1_score(class_IMDB_test_freq, test, average='micro')
  return f1_train, f1_valid, f1_test


In [102]:
%%time
from sklearn.dummy import DummyClassifier as DC
from sklearn.metrics import f1_score

uni_random_5 = DC(strategy = 'uniform')
majority_c_5 = DC(strategy = 'most_frequent')

print('Random Classifier = {}'.format(model_validation_FBoW_5(uni_random_5)))
# print('Majority Classifier = {}'.format(model_validation_FBoW_5(majority_c_5)))

for param in classifiers:
    clf_5 = param[0]
    param_grid_5 = param[1]
    print(clf_5)    
    best_clf_5 = optimize_parameters_FBoW(clf_5, param_grid_5, False)

Random Classifier = 0.5019
<class 'sklearn.naive_bayes.BernoulliNB'>
{'alpha': 0.001} --------> F1 = 0.8431000000000001
{'alpha': 0.002} --------> F1 = 0.8431000000000001
{'alpha': 0.003} --------> F1 = 0.8432
{'alpha': 0.004} --------> F1 = 0.8431000000000001
{'alpha': 0.005} --------> F1 = 0.8431000000000001
{'alpha': 0.006} --------> F1 = 0.8432
{'alpha': 0.007} --------> F1 = 0.8432
{'alpha': 0.008} --------> F1 = 0.8431000000000001
{'alpha': 0.009} --------> F1 = 0.8431000000000001
{'alpha': 0.01} --------> F1 = 0.8431000000000001
{'alpha': 0.02} --------> F1 = 0.8434
{'alpha': 0.05} --------> F1 = 0.8433
{'alpha': 0.1} --------> F1 = 0.843
Best params: {'alpha': 0.02}
Best F1 Score: 0.8434
 #################################################################################################### 

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
{'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 0.2, 'min_samples_split': 2} --------> F1 = 0.6393
{'cri

>> # **5.(c)**
Report training, validation, and test F1-measure for all the classifiers (with best
hyper-parameter configuration).

In [103]:
import re
from sklearn.tree import DecisionTreeClassifier

params_5 = []
with open("#5_b-All Parameters.txt") as f:
  for line in f:
    if 'Best params:' in line:
      params_5.append(line)
temp1_5 = repr(params_5[0])
temp1_5 = re.sub('Best params: ', '', temp1_5)
NB_best_param_5 = float(temp1_5.split()[1][:-4])
with open("#5_c-Best_Results.txt", 'a+') as g:
  g.write('NB_best alpha = {}\n'.format(NB_best_param_5))
print('NB_best alpha = {}'.format(NB_best_param_5))
#################################################


NB_best alpha = 0.02


In [104]:
temp2_5 = repr(params_5[1])
temp2_5 = re.sub('Best params: ', '', temp2_5)
criterion_5 = temp2_5.split()[1][:-1]
criterion_5 = criterion_5.replace("'",'')
max_dep_5 = [int(s) for s in temp2_5.split()[3] if s.isdigit()][0]
max_feat_5 = temp2_5.split()[5][:-1]
if max_feat_5 == str(None):
  max_feat_5 = None
else:
  max_feat_5 = max_feat_5.replace("'", '')
min_leaf_5 = float(temp2_5.split()[7][:-1])
mss_5 = int(temp2_5.split()[9][:-4])
tree_best_params_5 = [criterion_5, max_dep_5, max_feat_5, min_leaf_5, mss_5]
with open("#5_c-Best_Results.txt", 'a+') as g:
  g.write('[criterion, max_depth, max_features, min_samples_leaf, min_samples_split] = {}\n'.
          format(tree_best_params_5))
print('criterion, max_depth, max_features, min_samples_leaf, min_samples_split =', tree_best_params_5)
##################################################


criterion, max_depth, max_features, min_samples_leaf, min_samples_split = ['gini', 8, None, 0.2, 2]


In [105]:
temp3_5 = repr(params_5[2])
temp3_5 = re.sub('Best params: ', '', temp3_5)
c_5 = float(temp3_5.split()[1][:-1])
temp3_loss_5 = temp3_5.split()[3][:-4]
temp3_loss_5 = temp3_loss_5.replace("'","" )
svm_best_params_5 = [c_5, temp3_loss_5]
with open("#5_c-Best_Results.txt", 'a+') as g:
  g.write('[C, loss] = {}\n {}'.
          format(svm_best_params_5, seperator))
print('[C, loss] = ', svm_best_params_5)

[C, loss] =  [100.0, 'squared_hinge']


In [106]:
print('NB_best_params are {},\n tree_best_params are {},\n svm_best_params are {}.'.
      format(NB_best_param_5, tree_best_params_5, svm_best_params_5))



NB_best_params are 0.02,
 tree_best_params are ['gini', 8, None, 0.2, 2],
 svm_best_params are [100.0, 'squared_hinge'].


In [107]:
optimized_bayes_5 = BernoulliNB(alpha = NB_best_param_5)
optimized_tree_5 = DecisionTreeClassifier(criterion = tree_best_params_5[0],
                                        max_features = tree_best_params_5[2],
                                        max_depth = float(tree_best_params_5[1]),
                                        min_samples_leaf = tree_best_params_5[3],
                                        min_samples_split = tree_best_params_5[4], 
                                        random_state = 42)
optimized_svm_5 = svm.LinearSVC(C = svm_best_params_5[0], loss = svm_best_params_5[1], random_state = 42)


In [108]:
%%time

print('Optimal NB f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_5(optimized_bayes_5)))
print('Optimal Tree f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_5(optimized_tree_5)))
print('Optimal SVM f1 for train, valid, test = {}'.
      format(model_train_valid_test_FBoW_5(optimized_svm_5)))

f = open("#5_c-Best_Results.txt", 'a+')
f.write('\n NB_best_params are {}, \n tree_best_params are {}, \n svm_best_params are {} \n {}\n'.
        format(NB_best_param_5, tree_best_params_5, svm_best_params_5, seperator))
f.write('Optimal NB f1 for the (train, valid, test) = {},\n Optimal Tree f1 for train, valid, test = {}, \n Optimal SVM f1 for train, valid, test = {}\n {}'.
        format(model_train_valid_test_FBoW_5(optimized_bayes_5), 
               model_train_valid_test_FBoW_5(optimized_tree_5), 
               model_train_valid_test_FBoW_5(optimized_svm_5), 
               seperator))
f.close()

Optimal NB f1 for train, valid, test = (0.8725333333333334, 0.8434, 0.83572)
Optimal Tree f1 for train, valid, test = (0.6439333333333334, 0.6393, 0.64708)
Optimal SVM f1 for train, valid, test = (0.9467333333333332, 0.8795, 0.8746)
CPU times: user 56.5 s, sys: 4.07 s, total: 1min
Wall time: 55.9 s


>> # **5.(d)**
Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyperparameter that fetched you the best results.


>> # **5.(e)**
Compare the performance with the binary bag-of-words based classifiers. Why the
difference in performance? Give a brief explanation comparing BBoW Naive Bayes
and FBoW Naive Bayes and similarly for Decision Trees and Linear SVM.

>> # **5.(f)**
Which representation is better ? Why ?

>> # **5.(g)**
What did you observe about the relative performance of the classifiers when the
dataset is changed? What aspects of the dataset aided or hampered the performance
of that classifier.
